<a href="https://colab.research.google.com/github/snackasnack/Finanical-Indicator/blob/main/OBV-TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 19.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=2505e679b405781d12bf9b9a6c0b84f62f00f659317f2d7dac99fab368bf9a70
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
pip install -U -q PyDrive

In [ ]:
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
import glob,os
import urllib.request as ur
from io import StringIO
import requests

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import drive

yf.pdr_override()

In [ ]:
def get_ticker_list_gd(mktcap_min=0):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  stock_csv_link1 = "https://drive.google.com/file/d/19yzr67mg1FDJSasI8jzb24X6aFIRRu9-/view?usp=sharing"
  stock_csv_link2 = "https://drive.google.com/file/d/1JV7Kr4o1G8BVdEbnVC1wo_30pN4hXejG/view?usp=sharing"

  downloaded = drive.CreateFile({'id':'19yzr67mg1FDJSasI8jzb24X6aFIRRu9-'})
  downloaded.GetContentFile('nasdaq_screener_26012021.csv')
  stock_csv1 = pd.read_csv('nasdaq_screener_26012021.csv')

  downloaded = drive.CreateFile({'id':'1JV7Kr4o1G8BVdEbnVC1wo_30pN4hXejG'})
  downloaded.GetContentFile('nasdaq_screener_29012021.csv')
  stock_csv2 = pd.read_csv('nasdaq_screener_29012021.csv')

  stock_csv1 = stock_csv1[stock_csv1["Market Cap"]>mktcap_min]
  stock_csv2 = stock_csv2[stock_csv2["Market Cap"]>mktcap_min]

  nasdaq_lst = stock_csv1['Symbol'].tolist() 
  nyse_lst = stock_csv2['Symbol'].tolist()

  set_1 = set(nasdaq_lst)
  set_2 = set(nyse_lst)
  difference = list(set_2 - set_1)
  combined_lst = nasdaq_lst + difference
  return combined_lst

def get_ticker_list_link():
  link = "https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed-symbols_csv/data/595a1f263719c09a8a0b4a64f17112c6/nasdaq-listed-symbols_csv.csv"

  with ur.urlopen(link) as url:
    s = url.read()
    s = str(s, 'utf-8')
    data = StringIO (s)
    print(type(s))
    df = pd.read_csv(data)
    lst = df['Symbol'].tolist()
    print("The number of tickers retrieved is: ", len(lst))
    return lst

def get_stock(ticker):
  stock = pdr.get_data_yahoo(str(ticker))
  return stock

def calculate_obv(ticker, days):
  df = get_stock(ticker)
  truncated_df = df.iloc[-days:]
  
  pos_move = []
  neg_move = []
  OBV_Value = 0
  
  for count in range(days):
    if truncated_df.iloc[count,0] < truncated_df.iloc[count,3]:
      pos_move.append(count)
      #print("pos", pos_move)
    elif truncated_df.iloc[count,0] > truncated_df.iloc[count,3]:
      neg_move.append(count)
      #print("neg", neg_move)

  for i in pos_move:
    OBV_Value = round(OBV_Value + (truncated_df.iloc[i,5]/truncated_df.iloc[i,0]))
    #print(OBV_Value)
  for i in neg_move:
    OBV_Value = round(OBV_Value - (truncated_df.iloc[i,5]/truncated_df.iloc[i,0]))
    #print(OBV_Value)
  return OBV_Value

def create_df():
  df = pd.DataFrame(columns=["Stock","OBV_Value"])
  return df

def add_data(df, stock, OBV_Value):
  new_row = {"Stock": str(stock), "OBV_Value": int(OBV_Value)}
  new_df = df.append(new_row, ignore_index = True)
  print(stock, "added succcessfully")
  return new_df

def save_csv(table,name="10_days_OBV"):
  drive.mount('drive')
  table.reset_index(drop=True, inplace=True)
  table.to_csv('drive/My Drive/Google Colab/'+str(name)+'.csv')  

In [ ]:
def main(ticker_list, days=7):
  no_of_operations = len(ticker_list)
  print("Number of stocks to process: ", no_of_operations)

  failed_lst = []
  failed = 0
  success = 0

  OBV_table = create_df()

  for i in range(no_of_operations):
      try:
        OBV_Value = calculate_obv(ticker_list[i],days)
        OBV_table = add_data(OBV_table, ticker_list[i], OBV_Value)
        success += 1
      except: 
        failed_lst.append(ticker_list[i])
        failed += 1

  print (success, "number of stocks have been processed")
  print (failed, "number of stocks failed to be processed")
  print (failed_lst)

  OBV_table = OBV_table.sort_values(by=["OBV_Value"],ascending=False)
  return OBV_table

In [ ]:
if __name__ == "__main__":
  mktcap_min = 100000000
  days = 10
  ticker_list = get_ticker_list_gd(mktcap_min)
  table = main(ticker_list, days)
  save_csv(table, "20_days_OBV")
  print(table)

  my_position = ["BNGO", "OCGN", "NIO", "NNDM","TIGR","LKCO","EH","BIDU", "NOK", "AAL", "DIS", "CCIV", "IQ", "XPEV", "SDGR", "CPNG","KRKR","AMST"]
  table2 = main(my_position, days)
  save_csv(table2, "my_position")
  print(table2)

Streaming output truncated to the last 5000 lines.
[*********************100%***********************]  1 of 1 completed
EVOP added succcessfully
[*********************100%***********************]  1 of 1 completed
VCYT added succcessfully
[*********************100%***********************]  1 of 1 completed
GASS added succcessfully
[*********************100%***********************]  1 of 1 completed
NATR added succcessfully
[*********************100%***********************]  1 of 1 completed
ECOL added succcessfully
[*********************100%***********************]  1 of 1 completed
EQBK added succcessfully
[*********************100%***********************]  1 of 1 completed
SCPL added succcessfully
[*********************100%***********************]  1 of 1 completed
NYMT added succcessfully
[*********************100%***********************]  1 of 1 completed
EXLS added succcessfully
[*********************100%***********************]  1 of 1 completed
CORT added succcessfully
[********